# Relevant Packages

In [1]:
#Import packages
import IPython.display as ipd
import numpy as np
import pyaudio, wave
import math
import matplotlib.pyplot as plt
from numpy.matlib import repmat
from scipy.fftpack import dct

# MFCC PART

Here, I write three functions to get MFCC feature (39 dim) one is from project 2 another two functions are made of librosa and python_speech_features. 

In [2]:
def getMFCC2(wavename):#without normalization
    import numpy as np
    import scipy.io.wavfile as wav
    from python_speech_features import mfcc
    fs, audio = wav.read(wavename)
    feature_mfcc = mfcc(audio, samplerate=fs)
    mfcc=[]
    mfcc.append(np.hstack([feature_mfcc[0],feature_mfcc[0],feature_mfcc[0]]))
    for i in range(1,len(feature_mfcc)-1):
        delta=np.zeros(13)
        for j in range(13):
            delta[j]=feature_mfcc[i+1][j]-feature_mfcc[i-1][j]
        mfcc.append(np.hstack([feature_mfcc[i],delta]))
    mfcc.append(np.hstack([feature_mfcc[-1],feature_mfcc[-1],feature_mfcc[-1]]))

    for i in range(1,len(mfcc)-1):
        acc=np.zeros(13)
        for j in range(13):
            acc[j]=mfcc[i+1][13+j]-mfcc[i-1][13+j]
        mfcc[i]=np.hstack([mfcc[i],acc])
    mfcc=np.array(mfcc)
    return mfcc

In [3]:
def getMFCC3(wavename):#with normalization
    import librosa
    audio, sr = librosa.load(wavename,sr=None)
    mfccs = librosa.feature.mfcc(y=audio, sr=sr,n_mfcc=39)
    return mfccs.T

# Create Templates


There are total 100 wave files, 10 samples for digit 1-10. All these files are saved in the following format by Huangrui Chu: digit_0.wav, digit_1.wav.  digit could be 0,1,2,...9.

In [4]:
def create_templates(foldername,start_instance,end_instance):
    #we could also use this function for project 1,2,3 for testing our algorithm
    #INPUT:
        #foldername: the name of folder where Huangrui save the audio files
        #end-start:the number of instance we are expected to use as the templates.
    #OUTPUT:
        #templates: we can easily fetch the template for the corresponding digit using:
                #len(templates)%10+digit
    templates=[]
    max_template_length=0
    for i_th_instance in range(start_instance,end_instance):
        for digit in range(0,10):
            #print(str(digit )+'_'+str(i_th_instance)+'.wav')
            mfcc=getMFCC2(foldername+"/"+str(digit )+'_'+str(i_th_instance)+'.wav')
            if max_template_length<len(mfcc):
                max_template_length=len(mfcc)
            templates.append(mfcc)
    return templates,max_template_length

# 高斯函数部分

In [4]:

import numpy as np
from numpy import *
from sklearn.cluster import KMeans
from scipy import sparse
import scipy.io as sio
from scipy import signal
import wave
import math
import gParam#就是作者设置的参数， 算了 不管他
import copy


ModuleNotFoundError: No module named 'gParam'

In [2]:
import math
import copy
 
def pdf(m,v,x):
    '''计算多元高斯密度函数
    输入:
    m---均值向量 SIZE×1
    v---方差向量 SIZE×1
    x---输入向量 SIZE×1
    输出:
    p---输出概率'''
    test_v = np.prod(v,axis=0)
    test_x = np.dot((x-m)/v,x-m)
    p = (2*math.pi*np.prod(v,axis=0))**-0.5*np.exp(-0.5*np.dot((x-m)/v,x-m))
    return p


In [3]:
#class of mix info from KMeans
class mixInfo:
    """docstring for mixInfo"""
    def __init__(self):
        self.Cmean = []
        self.Cvar = []
        self.Cweight = []
        self.CM = []
class hmmInfo:
    '''hmm model param'''
    def __init__(self):
        self.init = [] #初始矩阵
        self.trans = [] #转移概率矩阵
        self.mix = [] #高斯混合模型参数
        self.N = 0 #状态数


In [ ]:
# class of gmm_hmm model
class gmm_hmm:
    def __init__(self):
        self.hmm = [] #单个hmm序列,
        self.gmm_hmm_model = [] #把所有的训练好的gmm-hmm写入到这个队列
        self.samples = [] # 0-9 所有的音频数据
        self.smplInfo = [] #这里面主要是单个数字的音频数据和对应mfcc数据
        self.stateInfo = [gParam.NPDF,gParam.NPDF,gParam.NPDF,gParam.NPDF]#每一个HMM对应len(stateInfo)个状态,每个状态指定高斯个数(3)
    def loadWav(self,pathTop):
        for i in range(gParam.NUM):
            tmp_data = []
            for j in range(gParam.NUM):
                wavPath = pathTop + str(i) + str(j) + '.wav'
                f = wave.open(wavPath,'rb')
                params = f.getparams()
                nchannels,sampwidth,framerate,nframes = params[:4]
                str_data = f.readframes(nframes)
                #print shape(str_data)
                f.close()
                wave_data = np.fromstring(str_data,dtype=short)/32767.0
                #wave_data.shape = -1,2
                #wave_data = wave_data.T
                #wave_data = wave_data.reshape(1,wave_data.shape[0]*wave_data.shape[1])
                #print shape(wave_data),type(wave_data)                
                tmp_data.append(wave_data)
            self.samples.append(tmp_data)
    #循环读数据,然后进行训练        
    def hmm_start_train(self):
        Nsmpls = len(self.samples)
        for i in range(Nsmpls):
            tmpSmplInfo0 = []
            n = len(self.samples[i])
            for j in range(n):
                tmpSmplInfo1 = sampleInfo()
                tmpSmplInfo1.set_smpl_wav(self.samples[i][j])
                tmpSmplInfo0.append(tmpSmplInfo1)
            #self.smplInfo.append(tmpSmplInfo0)
            print '现在训练第%d个HMM模型' %i
            hmm0 = self.trainhmm(tmpSmplInfo0,self.stateInfo)
            print '第%d个模型已经训练完毕' %i
            # self.gmm_hmm_model.append(hmm0)
    #训练hmm        
    def trainhmm(self,sample,state):
        K = len(sample)
        print '首先进行语音参数计算-MFCC'
        for k in range(K):
            tmp = getMFCC2(sample[k].smpl_wav)
            sample[k].set_smpl_data(tmp) # 设置MFCCdata
        hmm = self.inithmm(sample,state)
        pout = zeros((gParam.MAX_ITER_CNT,1))# 记录训练过程中的distance，如果某一个interator 和之前的变化不大，就停止。
        for my_iter in range(gParam.MAX_ITER_CNT):
            print '第%d遍训练' %my_iter
            hmm = self.baum(hmm,sample)
            for k in range(K):
                pout[my_iter,0] = pout[my_iter,0] + self.viterbi(hmm,sample[k].smpl_data[0])
            if my_iter > 0:
                if(abs((pout[my_iter,0] - pout[my_iter-1,0])/pout[my_iter,0]) < 5e-6):
                    print '收敛'
                    self.gmm_hmm_model.append(hmm)
                    return hmm
        self.gmm_hmm_model.append(hmm)
# init hmm
    def inithmm(self,sample,M):
        K = len(sample)# K是 一个词对应的sample 数量
        N0 = len(M)
        self.N = N0#######N0 是 state 数量
        #初始概率矩阵
        hmm = hmmInfo()
        hmm.init = zeros((N0,1))
        hmm.init[0] = 1
        hmm.trans = zeros((N0,N0))
        hmm.N = N0
        #初始化转移概率矩阵
        for i in range(self.N-1):
            hmm.trans[i,i] = 0.5
            hmm.trans[i,i+1] = 0.5
        hmm.trans[self.N-1,self.N-1] = 1
        #概率密度函数的初始聚类
        #分段
        for k in range(K):
            T = len(sample[k].smpl_data[0])
            #seg0 = []
            seg0 = np.floor(arange(0,T,1.0*T/N0))
            #seg0 = int(seg0.tolist())
            seg0 = np.concatenate((seg0,[T]))
            #seg0.append(T)
            sample[k].seg = seg0
        #对属于每个状态的向量进行K均值聚类,得到连续混合正态分布
        mix = []#是储存了这个word的，所有 state 对应的 guassian distribution数量的，一个高斯混合模型参数
        for i in range(N0):
            vector = []#被归属当前state的vector
            for k in range(K):
                seg1 = int(sample[k].seg[i])
                seg2 = int(sample[k].seg[i+1])
                tmp = []
                tmp = sample[k].smpl_data[0][seg1:seg2][:]
                if k == 0:
                    vector = np.array(tmp)
                else:
                    vector = np.concatenate((vector, np.array(tmp)))
                #vector.append(tmp)
            # tmp_mix = mixInfo()
            # print id(tmp_mix)
            tmp_mix = self.get_mix(vector,M[i],mix)
            # mix.append(tmp_mix)
        hmm.mix = mix
        return hmm
    # get mix data
    def get_mix(self,vector,K,mix0):
        kmeans = KMeans(n_clusters = K,random_state=0).fit(np.array(vector))
        #计算每个聚类的标准差,对角阵,只保存对角线上的元素
        mix = mixInfo()
        var0 = []
        mean0 = []
        #ind = []
        for j in range(K):
            #ind = [i for i in kmeans.labels_ if i==j]
            ind = []
            ind1 = 0
            for i in kmeans.labels_:
                if i == j:
                    ind.append(ind1)
                ind1 = ind1 + 1
            tmp = [vector[i][:] for i in ind]
            var0.append(np.std(tmp,axis=0))
            mean0.append(np.mean(tmp,axis=0))
        weight0 = zeros((K,1))
        for j in range(K):
            tmp = 0
            ind1 = 0
            for i in kmeans.labels_:
                if i == j:
                    tmp = tmp + ind1
                ind1 = ind1 + 1
            weight0[j] = tmp
        weight0=weight0/weight0.sum()
        mix.Cvar = multiply(var0,var0)
        mix.Cmean = mean0
        mix.CM = K
        mix.Cweight = weight0
        mix0.append(mix)
        return mix0
    #baum-welch 算法实现函数体
    def baum(self,hmm,sample):
        mix = copy.deepcopy(hmm.mix)#高斯混合
        N = len(mix)  #HMM状态数
        K = len(sample) #语音样本数
        SIZE = shape(sample[0].smpl_data[0])[1]    #参数阶数,MFCC维数
        print '计算样本参数.....'
        c = []
        alpha = []
        beta = []
        ksai = []
        gama = []
        for k in range(K):
            c0,alpha0,beta0,ksai0,gama0 = self.getparam(hmm, sample[k].smpl_data[0])
            c.append(c0)
            alpha.append(alpha0)
            beta.append(beta0)
            ksai.append(ksai0)
            gama.append(gama0)
        # 重新估算概率转移矩阵
        print '----- 重新估算概率转移矩阵 -----'
        for i in range(N-1):
            denom = 0
            for k in range(K):
                ksai0 = ksai[k]
                tmp = ksai0[:,i,:]#ksai0[:][i][:]
                denom = denom + sum(tmp)
            for j in range(i,i+2):
                norm = 0
                for k in range(K):
                    ksai0 = ksai[k]
                    tmp = ksai0[:,i,j]#[:][i][j]
                    norm = norm + sum(tmp)
                hmm.trans[i,j] = norm/denom
        # 重新估算发射概率矩阵,即GMM的参数
        print '----- 重新估算输出概率矩阵,即GMM的参数 -----'
        for i in range(N):
            for j in range(mix[i].CM):
                nommean = zeros((1,SIZE))
                nomvar = zeros((1,SIZE))
                denom = 0
                for k in range(K):
                    gama0 = gama[k]
                    T = shape(sample[k].smpl_data[0])[0]
                    for t in range(T):
                        x = sample[k].smpl_data[0][t][:]
                        nommean = nommean + gama0[t,i,j]*x
                        nomvar = nomvar + gama0[t,i,j] * (x - mix[i].Cmean[j][:])**2
                        denom = denom + gama0[t,i,j]
                hmm.mix[i].Cmean[j][:] = nommean/denom
                hmm.mix[i].Cvar[j][:] = nomvar/denom
                nom = 0
                denom = 0
                #计算pdf权值
                for k in range(K):
                    gama0 = gama[k]
                    tmp = gama0[:,i,j]
                    nom = nom + sum(tmp)
                    tmp = gama0[:,i,:]
                    denom = denom + sum(tmp)
                hmm.mix[i].Cweight[j] = nom/denom
        return hmm
                
    #前向-后向算法
    def getparam(self,hmm,O):
        '''给定输出序列O,计算前向概率alpha,后向概率beta
        标定系数c,及ksai,gama
        输入: O:n*d 观测序列
        输出: param: 包含各种参数的结构'''
        T = shape(O)[0]
        init = hmm.init #初始概率
        trans = copy.deepcopy(hmm.trans) #转移概率
        mix = copy.deepcopy(hmm.mix) #高斯混合
        N = hmm.N #状态数
        #给定观测序列,计算前向概率alpha
        x = O[0][:]
        alpha = zeros((T,N))
        #----- 计算前向概率alpha -----#
        for i in range(N): #t=0
            tmp = hmm.init[i] * self.mixture(mix[i],x)
            alpha[0,i] = tmp #hmm.init[i]*self.mixture(mix[i],x)
        #标定t=0时刻的前向概率
        c = zeros((T,1))
        c[0] = 1.0/sum(alpha[0][:])
        alpha[0][:] = c[0] * alpha[0][:] 
        for t in range(1,T,1): # t = 1~T
            for i in range(N):
                temp = 0.0
                for j in range(N):
                    temp = temp + alpha[t-1,j]*trans[j,i]
                alpha[t,i] = temp *self.mixture(mix[i],O[t][:])
            c[t] = 1.0/sum(alpha[t][:])
            alpha[t][:] = c[t]*alpha[t][:]
 
        #----- 计算后向概率 -----#
        beta = zeros((T,N))
        for i in range(N): #T时刻
            beta[T-1,i] = c[T-1]
        for t in range(T-2,-1,-1):
            x = O[t+1][:]
            for i in range(N):
                for j in range(N):
                    beta[t,i] = beta[t,i] + beta[t+1,j]*self.mixture(mix[j],x) * trans[i,j]
            beta[t][:] = c[t] * beta[t][:]
        # 过渡概率ksai
        ksai = zeros((T-1,N,N))
        for t in range(0,T-1):
            denom = sum(np.multiply(alpha[t][:],beta[t][:]))
            for i in range(N-1):
                for j in range(i,i+2,1):
                    norm = alpha[t,i]*trans[i,j]*self.mixture(mix[j],O[t+1][:])*beta[t+1,j]
                    ksai[t,i,j] = c[t]*norm/denom
        # 混合输出概率 gama
        gama = zeros((T,N,max(self.stateInfo)))
        for t in range(T):
            pab = zeros((N,1))
            for i in range(N):
                pab[i] = alpha[t,i]*beta[t,i]
            x = O[t][:]
            for i in range(N):
                prob = zeros((mix[i].CM,1))
                for j in range(mix[i].CM):
                    m = mix[i].Cmean[j][:]
                    v = mix[i].Cvar[j][:]
                    prob[j] =  mix[i].Cweight[j] * pdf(m,v,x)
                    if mix[i].Cweight[j] == 0.0:
                        print pdf(m,v,x)
                tmp = pab[i]/pab.sum()
                tmp = tmp[0]
                temp_sum = prob.sum()
                for j in range(mix[i].CM):
                    gama[t,i,j] = tmp*prob[j]/temp_sum
        return c,alpha,beta,ksai,gama                
    def mixture(self,mix,x):
        '''计算输出概率
        输入:mix--混合高斯结构
        x--输入向量 SIZE*1
        输出: prob--输出概率'''        
 
        prob = 0.0
        for i in range(mix.CM):
            m = mix.Cmean[i][:]
            v = mix.Cvar[i][:]
            w = mix.Cweight[i]
            tmp = pdf(m,v,x)
            #print tmp
            prob = prob + w * tmp #* pdf(m,v,x)
        if prob == 0.0:
            prob = 2e-100
        return prob
    #维特比算法
    def viterbi(self,hmm,O):
        '''%输入:
        hmm -- hmm模型
        O   -- 输入观察序列, N*D, N为帧数,D为向量维数
        输出:
        prob -- 输出概率
        q    -- 状态序列
        '''
        init = copy.deepcopy(hmm.init)
        trans = copy.deepcopy(hmm.trans)#hmm.trans
        mix = hmm.mix
        N = hmm.N
        T = shape(O)[0]
        #计算Log(init)
        n_init = len(init)
        for i in range(n_init):
            if init[i] <= 0:
                init[i] = -inf
            else:
                init[i]=log(init[i])
        #计算log(trans)
        m,n = shape(trans)
        for i in range(m):
            for j in range(n):
                if trans[i,j] <=0:
                    trans[i,j] = -inf
                else:
                    trans[i,j] = log(trans[i,j])
        #初始化
        delta = zeros((T,N))
        fai = zeros((T,N))
        q = zeros((T,1))
        #t=0
        x = O[0][:]
        for i in range(N):
            delta[0,i] = init[i] + log(self.mixture(mix[i],x))
        #t=2:T
        for t in range(1,T):
            for j in range(N):
                tmp = delta[t-1][:]+trans[:][j].T
                tmp = tmp.tolist()
                delta[t,j] = max(tmp)
                fai[t,j] = tmp.index(max(tmp))
                x = O[t][:]
                delta[t,j] = delta[t,j] + log(self.mixture(mix[j],x))
        tmp = delta[T-1][:]
        tmp = tmp.tolist()
        prob = max(tmp)
        q[T-1]=tmp.index(max(tmp))
        for t in range(T-2,-1,-1):
            q[t] = fai[t+1,int(q[t+1,0])]
        return prob